In [1]:
import pandas as pd
import re
import unicodedata
import html2text
from sqlalchemy import create_engine

In [ ]:
# read table from DB
def read_table(user = 'postgres', password = 'secret', 
                db_name = 'db_name', table_name = 'tb_name', chunksize = 1000):

    db_url = 'postgresql://localhost/{}?user={}&password={}'.format(db_name, user, password)
    sql_engine = create_engine(db_url, echo=False)
    conn = sql_engine.connect()

    return pd.read_sql_table(table_name, conn, chunksize=chunksize)

In [ ]:
def get_text_maker():
    # HTML2Text keeps formatting, so all paragraphes are saved
    # you get markdown text, so headers are marked with '#' and lists with '*'
    text_maker = html2text.HTML2Text()
    # Do not include any formatting for links
    text_maker.ignore_links = True
    # Do not include any formatting for images
    text_maker.ignore_images = True
    # for using unicode
    text_maker.unicode_snob = True
    text_maker.body_width = 0
    text_maker.single_line_break = True
    # Ignore all emphasis formatting in the html.
    text_maker.ignore_emphasis = True

    return text_maker

In [ ]:
# process chunks of table from DB
def get_news_texts(output_file)
    text_maker = get_text_maker()
    table_chunks = read_table()
    df_parts = []
    for chunk in table_chunks:
        chunk['text'] = chunk.text.apply(lambda x: text_maker.handle(x) if pd.notnull(x) else None)
        df_parts.append(chunk)
        del chunk
    df = pd.concat(df_parts)
    df = df.reset_index(drop=True)
    df.to_csv(output_file, index=False)
    return df

In [ ]:
# read table from file
news = pd.read_csv('path/to/file.csv')
text_maker = get_text_maker()
news['text'] = news.text.apply(lambda x: text_maker.handle(x) if pd.notnull(x) else None)

In [ ]:
# based on https://github.com/brown-uk/nlp_uk/blob/master/src/main/groovy/org/nlp_uk/other/CleanText.groovy

lat_to_cyr_map = {
    'a': 'а',
    'c': 'с',
    'e': 'е',
    'i': 'і',
    'l': 'І',
    'o': 'о',
    'u': 'и',
    'p': 'р',
    'n': 'п',
    'm': 'т',
    'x': 'х',
    'y': 'у',
    'k': 'к',
    'b': 'ь',
    'r': 'г',
    'A': 'А',
    'B': 'В',
    'C': 'С',
    'E': 'Е',
    'H': 'Н',
    'I': 'І',
    'K': 'К',
    'M': 'М',
    'O': 'О',
    'P': 'Р',
    'T': 'Т',
    'X': 'Х',
    'Y': 'У',
    "á": "а́",
    "Á": "А́",
    "é": "е́",
    "É": "Е́",
    "í": "і́",
    "Í": "І́",
    "ḯ": "ї́",
    "Ḯ": "Ї́",
    "ó": "о́",
    "Ó": "О́",
    "ú": "и́",
    "ý": "у́",
    "Ý": "У́",
    "0": "о"
}

cyr_to_lat_map = {}
for k, v in lat_to_cyr_map.items():
    cyr_to_lat_map[v] = k

APOSTROPHY_LIKE = ('”',
                   '‟',
                   '"',
                   '‘',
                   '′',
                   '\u0313',
                   '΄',
                   '’',
                   '´',
                   '`',
                   '’',
                   '?',
                   '*',
                   )
APOSTROPHY_PREFIX = 'бвгґдзкмнпрстфхш'
APOSTROPHY_SUFFIX = 'єїюя'


def remove_title_from_text(title, text):
    if pd.notnull(title) and pd.notnull(text):
        try:
            title = re.escape(title)
            text = re.sub(title, '', text)
        except:
            print(title)
    return text


def remove_newlines(text):
    text = re.sub(r"\n \n", "\n", text)
    text = re.sub(r"\n{2,}", "\n", text)
    text = re.sub(r" {2,}", " ", text)
    text = text.strip()
    return text


def clean(text):
    if pd.notnull(text):
        text = unicodedata.normalize("NFKC", text)
        text = re.sub(r"\\n", "\n", text)
        text = re.sub(r"\n{2,}", "\n", text)
        text = re.sub(r" {2,}", " ", text)
        text = text.strip()

        # change strange apostrophe to '
        text = re.sub(r"([{prefix}])[{apostrophy}]([{suffix}])".format(
            prefix=APOSTROPHY_PREFIX, apostrophy=''.join(APOSTROPHY_LIKE), suffix=APOSTROPHY_SUFFIX),
            r"\1'\2", text, flags=re.IGNORECASE)
        text = re.sub(r"([{prefix}])&#39\s?([{suffix}])".format(
            prefix=APOSTROPHY_PREFIX, suffix=APOSTROPHY_SUFFIX), r"\1'\2", text)

        # add space between sentences if needed (with workaround for  Цензор.НЕТ and ZN.UA)
        text = re.sub(r"([\.\?\!])(?!(NET|UA|НЕТ))([А-ЯІЇЄҐA-Z])", r"\1 \2\3", text)
       
        # clean up latin/cyrillic character mix
        # cases:
        # - latin symbols that look like cyrillic in ukrainian words
        # - cyrillic symbols that look like latin in english words
        text = re.sub(
            r"([бвгґдєжзийклмнптфцчшщьюяБГҐДЄЖЗИЙЛПФХЦЧШЩЬЮЯ]['’ʼ]?)([aceiopxyunmkbr0ABCEHIKMOPTXYáÁéÉíÍḯḮóÓúýÝ])",
            lambda x: x.group(1) + lat_to_cyr_map[x.group(2)], text)

        text = re.sub(
            r"([aceiopxyaceiopxyunmkbr0ABCEHIKMOPTXYáÁéÉíÍḯḮóÓúýÝ])(['’ʼ]?[бвгґдєжзийклмнптфцчшщьюяБГҐДЄЖЗИЙЛПФХЦЧШЩЬЮЯ])",
            lambda x: lat_to_cyr_map[x.group(1)] + x.group(2), text)

        text = re.sub(r"([bdfghjklmnrstuvwzDFGJLNQRSUVWZ]['’ʼ]?)([асеіорхуАВСЕНІКМНОРТХУ])",
                      lambda x: x.group(1) + cyr_to_lat_map[x.group(2)], text)

        text = re.sub(r"([асеіорхуАВСЕНІКМНОРТХУ])(['’ʼ]?[bdfghjklmnrstuvwzDFGJLNQRSUVWZ])",
                      lambda x: cyr_to_lat_map[x.group(1)] + x.group(2), text)

        text = re.sub(r"([а-яіїєґА-ЯІЇЄҐ]['’ʼ]?)([aceiopxyunmkbr0ABCEHIKMHOPTXYáÁéÉíÍḯḮóÓúýÝ])(['’ʼ]?[а-яіїєґА-ЯІЇЄҐ])",
                      lambda x: x.group(1) + lat_to_cyr_map[x.group(2)] + x.group(3), text)

        text = re.sub(r"([a-zA-Z]['’ʼ]?)([асеіорхуАВСЕНІКМНОРТХУ])(['’ʼ]?[a-zA-Z])",
                      lambda x: x.group(1) + cyr_to_lat_map[x.group(2)] + x.group(3), text)

        text = re.sub(r"([а-яіїєґ]\W{0,2} )([ayico])( [А-ЯІЇЄҐа-яіїєґ])",
                      lambda x: x.group(1) + lat_to_cyr_map[x.group(2)] + x.group(3), text)

        text = re.sub(r"([AIYBKOl])( [А-ЯІЇЄҐа-яіїєґ])",
                      lambda x: lat_to_cyr_map[x.group(1)] + x.group(2), text)

    return text


def text_cleaning(df):
    df['text'] = df.text.apply(clean)
    df['title'] = df.title.apply(clean)
    df['subtitle'] = df.subtitle.apply(clean)
    df['text'] = df.apply(lambda row: remove_title_from_text(row.title, row.text), axis=1)
    return df


In [ ]:
news = text_cleaning(news)

In [2]:
strip_patterns = {
#     parts to initially strip in all news 
    '': [
        (r"\*\[25 лютого\]: за юліанським календарем.*", re.S),
        (r"^[\w ]*фото:.*?\n", re.M | re.I),
        (r"Теги:.*", re.S),
        (r"Читайте також( статтю)?: ?\n?.*\n?", re.M | re.I),
        (r"Читайте більше:.*$", re.M),
        (r"Тако?же? читайте.*$", re.M),
        (r"Якщо (ви )?(побачили|виявили|помітили|знайшли)( орфографічну)? помилку.*", re.S|re.I), 
        (r"Если Вы заметили орфографическую ошибку.*", re.S), 
        (r"Если вы нашли ошибку в тексте.*", re.S),
#         (r"(^>.*\n)+", re.M), 
        (r"этот материал доступен на русском", re.I),
        (r"читать ((новость|этот материал) )?на русском", re.I),
        (r"Цей матеріал можна прочитати і російською мовою\n", re.I),
        (r"^Реклама$", re.M),
        (r"(ПУБЛИКАЦИИ|МАТЕРИАЛЫ) ПО ТЕМЕ.*", re.S),
        (r"^Твитнуть$", re.M),
        (r"^[ \*]*Розсилка ?\nВІДПРАВИТИ\n?\n?\n?", re.S|re.M),
        (r"\n\n\nКопіювати код для вставки", re.S), 
        (r"^Цей матеріал також доступний українською$", re.M), 
        (r"^Нажмите на фото для увеличения изображения.*$", re.M),
        (r"https://[a-z\./0-9@:%_\+~#\?&=\-\(\)]+", re.I|re.M),
        (r"фото: instagram.com", re.M),
        (r"фото:? (ua\.)?depositphotos(\.com)?", re.M|re.I),
        (r"© president\.gov\.ua ?$", re.M),
        (r" ?\* ?Поділитися (на|у) (Facebook|Twitter|Telegram|ВКонтакте)$", re.M),
        (r"Share on .*$", re.M), 
        (r" ?\* ?Надіслати листом$", re.M),
        (r"^© ?REUTERS ?$", re.M),
        (r"^[\* ]*Кількість (переглядів|коментарів) \d* ?$", re.M),
        (r"^© фото president\.gov\.ua ?$", re.M),
        (r"Авторские права на изображение: TUT. BY", re.M),
        (r"^Коментувати Роздрукувати$", re.M),
        (r"^(Коментувати|Поділитись|Рекомендую|Підписатися)[\: ]*$", re.M),
        (r"Фото: ?Pixabay", re.I),
        (r"Share to (facebook|twitter|telegram|viber)", re.I),
        (r"Найцікавіші історії та новини дня тепер в Telegram!", re.M),
        (r"Арх(и|і)вно?е фото\n", re.I|re.M),
        (r"фото з відкритих джерел\n", re.I|re.M),
        (r"^З відкритих джерел\n", re.I|re.M),
        (r"фото reuters", re.I|re.M),
        (r"[ \*]*Tweet", re.M),
        (r"^Загрузка[\.]*\n", re.M),
        (r"^Реклама$", re.M)

    ],
    
     # strip patterns by domains 
    
    'apostrophe.ua': [
        (r"### Новини.*?Всі новини", re.S),
        (r"### Головні новини.*?### Новини\n### \d+ \w{4,10}\n", re.S),
        (r'^\d\d:\d\d\n.*\n', re.M),
        (r"Версія для друку.*", re.S),
        (r"\w+ новини тепер в Telegram.*", re.S),
        (r"^Читайте тако?же?.*$", re.M),
        (r"Підписуйтесь на канал.*", re.S),
        (r"^Поделиться$", re.M),
        (r"^1 \/ 1$", re.M),
        (r"Як повідомляв \"Апостроф\".*", re.M)
    ],
    'https://www.rbc.ua': [
        (r"Читайте нас в Google News та підписуйтесь.*", re.S),
        (r"Надіслати новину другу\nКому\:\*\nВаш коментар\nНадіслати(\nДякуємо, Ваше повідомлення відправлено\!  \n  \nOk\n)?", re.M),
        (r"Дякуємо, Ваше повідомлення відправлено\!", re.M),
        (r"^Ok$", re.M)
    ],
    'https://www.unian.ua': [
        (r"Читайте останні новини України та світу.*", re.S),
        (r"^Читайте новини шоу\-бізнесу.*", re.M),
        (r"Якщо ви знайшли помилку.*", re.S),
        (r"^Читайте тако?же?.*$", re.M),
        (r"Підписуйтесь на канал УНІАН.*", re.S),
        (r"^ ?#+ ?Що було раніше[ \:]*\n(^ *\*.*\n)+", re.M),
        (r"^[ #]*Що цьому передувало ?\n.*", re.S|re.M),
        (r"^[ #]*((Новини по темі)|(Вас (також )?можуть зацікавити (інші )?новини)|(Вам (також )?може бути цікаво))[ \:]*\n[ \*]+.*", re.S|re.M),
        (r"^ ?#+ ?Більше новин(.*?)\n[ \*]+.*", re.S|re.M),
        (r"^ ?#+ ?Коронавірус в Україні( та світі)?\: (((важливі|інші|останні) новини)|(що відомо)|(інші новини за темою)).*", re.S|re.M),
        (r"^ ?#+ ?(.*?): новини на тему[ \:]*\n[ \*]+.*", re.S|re.M),
        (r"^ ?#+ ?(.*?): що варто знати(.*?)\n[ \*]+.*", re.S|re.M),
        (r"^[ #]*Інші новини(.*?)\n[ \*]+.*", re.S|re.M)
        
    ],
    'https://censor.net.ua': [
        (r"^.*?Редактор Цензор\. НЕТ", re.S|re.M),
        (r"Мне нравится.*", re.S),
        (r"^Читайте (также ?)?(на \"?Цензор\. НЕТ\"?)?:.*?$", re.M),
#         (r"^Все про:.*?$", re.M),
        (r"Топ комментарии.*", re.S),
        (r"^Читайте:.*$", re.M),
        (r"\nИсточник:.*", re.S)
    ],
    'www.unn.com.ua': [
        (r"(Що було раніше)|(Варто нагадати).*", re.S),
    ],
    'https://politeka.net/uk': [
        (r"Популярні новини зараз.*?Показати ще", re.S),
        (r"^Як повідомляла Politeka.*", re.M),
        (r"^Також Politeka.*", re.M)
    ],
    'www.radiosvoboda.org': [
        (r"^ДИВІТЬСЯ ТАКОЖ: ?\n#+.*?$", re.M),
        (r"[\# ]*### FACEBOO. КОМЕНТАРІ.*", re.S),
        (r"Отримати Adobe Flash Player.*\* \d+p | \d+(\,?\d+)?MB", re.S),
        (r"[\# ]*Читай нас в Google News.*", re.S)
    ],
    'https://dt.ua/': [
        (r"(\^* #[\w ]+)+", re.M)
    ],
    'https://hromadske.ua/': [
        (r"Поділитись.*", re.S),
#         (r"Більше про:.*", re.S),
        (r"^Інформаційна гігієна не менш важлива за особисту.*?$", re.M),
        (r"^[# ]*читайте також *\n.*\n.*\n", re.M), ###???
        (r"^[# ]*читайте також *\n.*$", re.M)
    ],
    'https://www.obozrevatel.com': [
        (r"Тисни! Підписуйся!.*", re.S),
        (r"Ти ще не підписаний на наш Telegram.*", re.S),
        (r"Не набридаємо! Тільки найважливіше.*", re.S),
        (r"Підписуйся на наш Telegram.*", re.S),
        (r"^Читайте:.*?$", re.M | re.I),
        (r"Як повідомляв раніше OBOZREVATEL.*", re.M),
        (r"^Як повідомляв OBOZREVATEL:\n([ \*]*.*\n)+", re.M|re.I)
    ],
    'https://www.segodnya.ua/ua': [
        (r"^Також дивіться.*?$", re.M),
        (r"Підпишись на наш telegram.*", re.S),
        (r"[\# ]*Ще кілька матеріалів по темі:.*", re.S),
        (r"^[\# ]*В тренді\n.*$", re.M)

    ],
    'focus.ua': [
        (r"^Читайте также.*$", re.M)
#         (r"Как ранее сообщал Фокус:.*", re.S)
    ],
    '112.ua':[
        (r"^#\w+.*?#\w+.*\n", re.M),
        (r"^Новини за темою:.*?$", re.M),
        (r"^(Як|Раніше) повідомляли\,.*", re.M)
    ],
    'https://www.liga.net': [
        (r"^Читайте нас в Telegram:.*?$", re.M),
        (r"Еще по теме.*", re.S),
        (r"Ще по темі.*", re.S),
        (r"^Читайте также.*\n", re.M),
        (r"Присоединяйтесь к Instagram Liga.*", re.S),
        (r"^Підписуйтесь на.*$", re.M)
    ],
    'espreso.tv':[
        (r"Слідкуйте за подіями в Україні та світі.*", re.S),
        (r"Все по темі.*", re.S),
        (r"Підписуйтесь на Telegram.*", re.S),
#         (r"(^[\* ]*.*\n)+", re.M)
    ],
    'strana.ua': [
        (r"^Читайте также ?\n( ?\*.*?\n)+", re.M),
        (r"Подпишитесь на телеграм-канал.*", re.S),
        (r"Читайте Страну в Google News.*", re.S),
        (r'Ранее \"Страна\" сообщала\,.*', re.M)
        
    ],
    'tsn.ua': [
        (r"^\* Facebook.*\* Відправити лист", re.S|re.M),
        (r"^\* WhatsApp.*\* Відправити лист", re.S|re.M),
        (r"\* Відправити лист", re.S),
        (r"\* Повноекранний режим$", re.M),
        (r"[# ]*Читайте також: *\n.*\n", re.M),
        (r"Приєднуйтесь також до tsn\.ua у Google News.*", re.S),
         (r"^ ?\* ?(Facebook|Twitter|Telegram|Messenger|Viber)$", re.M),
        (r"^ ?\* ?WhatsApp$", re.M),
    ],
    'https://hromadske.radio': [
        (r"Підписуйтесь на Telegram-канал.*", re.S)
    ],
    'znaj.ua': [
        (r"Обов'язково підпишись.*", re.S),
        (r"Популярні новини зараз.*Показати ще", re.S),
        (r"Популярні новини.*", re.S),
        (r"^Підпишіться на Знай.*", re.M),
        (r"^Дивіться:.*", re.M),
        (r"^Читайте Знай в Google News!\n", re.M),
        (r"^((Як повідомляв(портал)? (.*?)Знай)|(Також (.*?)Знай.{,5} (писав|повідомл))|(Раніше ми повідомляли)).*", re.M),
        (r"^Нещодавно Знай передавав.*", re.M),

    ],
    'fakty.com.ua': [
        (r"^Читайте:.*?$", re.M),
        (r"^Читайте\n.*\n", re.M)
    ],
    'https://www.epravda.com.ua': [
        (r"Читайте детальніше:.*", re.S),
        (r"Читайте нас у Telegram.*", re.S),
        (r"Приєднуйтесь до нашого каналу на YouTube.*", re.S),
        (r"^(Нагад(у|а)ємо|Передісторія):.*", re.S|re.M),
        (r"^Що було раніше\:.*", re.S|re.M),
        (r"Довідково:.*", re.M)
        
    ],
    'https://www.pravda.com.ua': [
        (r"Читайте детальніше:.*", re.S),
        (r"Читайте нас у Telegram.*", re.S),
        (r"^Українська правда$", re.M),
        (r"Приєднуйтесь до нашого каналу на YouTube.*", re.S),
        (r"^(Нагад(у|а)ємо|Передісторія): *\n(^ *\*.*\n)+", re.M),
        (r"^Що було раніше\:.*", re.S|re.M),
        (r"Довідково:.*", re.M)
        
    ],
    'https://ukr.lb.ua': [
        (r"Читайте головні новини LB\.ua.*", re.S),
        (r"^(Facebook|Twitter)$", re.M),
        (r"^Темы:.*", re.S|re.M)
    ],
    'https://ukranews.com': [
        (r"Больше новостей о:.*", re.S),
        (r"Коментарии Facebook.*", re.S),
        (r"^\* \d+\n \* \d+\n", re.S|re.M),
        (r"^Українськ. Новини\nУкраїнськ. Новини\n.*\n\#.*\n.*\n", re.M),
        (r"^Українськ. Новини\n.*\n\#.*\n.*\n", re.M),
        (r"Читайте также:.*$", re.M),
        (r"^Как сообщали Українськ. Новини.*", re.M),
        (r"Добавьте ukranews\.com.*", re.S),
        (r"Подпишитесь на (рассылку|авторский).*", re.S) 
    ],
   
    'zik.ua': [
        (r"^Новини за темою:.*?$", re.M),
        (r"^(Як|Раніше) повідомляли\,.*", re.M)

    ],
    'https://ua.korrespondent.net/': [
        (r"(Новини|Новости) (від|от) Корреспондент.*", re.S),
        (r"(Підписуйтесь|Подписывайтесь) на наш канал.*", re.S),
        (r"^Напередодні повідомлялося\,.*", re.M)
       
    ],
    'vgolos.com.ua': [
        (r"###? Як писав \“Вголос\”:.*", re.S),
        (r"^Всі свіжі новини на нашомуTelegram\-каналі Приєднуйся!$", re.M),
        (r"[\# ]+Читайте більше.*", re.S)
    ],
    'glavcom.ua': [
        (r"## Хроника событий.*", re.S),
        (r"Опитування$", re.M),
        (r"Читайте также:.*$", re.M),
        (r"Читайте також ?: ?\n?( \* .*\n?){1,}", re.M),
        (r"^на весь екран згорнути$", re.M)
    ],
    '24tv.ua': [
        (r"Читайте тако?же?:?.*$", re.M),
        (r"^Поділитися новиною ?$", re.M),
        (r"^ ?\* \* \* \* \* ?$", re.M),
        (r"^[# ]*Що було раніше[ \:]*\n(^ *\*.*\n)+", re.M)
    ], 
    'nv.ua': [
        (r"^Хроники коронавируса.*Отправляем ежедневно в 08:00", re.S|re.M),
        (r"^Читайте также:\n\n.*\n", re.M)
    ],
    'suspilne.media': [
        (r"## Що відомо.*", re.S),
        (r"## Читайте також.*", re.S),
        (r"(> )?Читайте також:.*", re.M)
    ],
    'babel.ua': [
        (r"(^ \* .*$){1,}$", re.M)
    ],
    'bykvu.com': [
        (r"(^ *\* *.*$){1,}$", re.M)
    ],
    'vesti.ua': [
        (r"Подпишитесь на ежедневную еmail-рассылку.*", re.S),
        (r"^Как сообщалось ранее.*", re.M)
    ],
    'fakty.ua': [
        (r"Читайте нас в Telegram-канале.*", re.S),
        (r"Сделайте \"ФАКТЫ\".*", re.S)
    ],
    'kp.ua': [
        (r"[# ]*(ЧИТАЙТЕ|ЧИАТЙТЕ) ТАКЖЕ.*", re.S)
    ],
    # this shoud be the last part
    # patterns to finally strip in all news 
    # it is expected that every domain contains a dot
    '.': [
        (r"^[#\* ]*Читайте тако?же? *:? *\n?.*$", re.M | re.I),
        (r"^[#\* ]*Тако?же? читайте *:? *\n?.*$", re.M | re.I),
        (r"^[#\* ]*(Нагад(а|у)ємо|Напомним)\,.*", re.M),
        (r"^((Ранее мы сообщали)|(Также сообщалось)|(Раніше повідомляли))\,.*", re.M),
        (r"Раніше ми розповідали\,.*", re.M),
        (r"^[\-\*#\| ]+\n", re.M)
    ]

}


def mystrip(df):
    for domain, patterns in strip_patterns.items():
        print(domain)
        domain_mask = df.domain.str.contains(domain)
        for part_to_strip in patterns:
            df.text.update(df[domain_mask].text.str.replace(part_to_strip[0], "", flags=part_to_strip[1]))
    df['text'] = df.text.apply(remove_newlines)

In [ ]:
mystrip(news)

In [ ]:
alias = {"https://www.pravda.com.ua": "Українська правда",
    "strana.ua": "Страна.ua",
    "https://www.rbc.ua": "РБК-Україна",
    "https://www.obozrevatel.com": "Обозреватель",
    "24tv.ua": "24 канал",
    "https://censor.net.ua": "Цензор.НЕТ",
    "tsn.ua": "ТСН",
    "gordonua.com": "Гордон",
    "https://politeka.net/uk": "Politeka",
    "https://www.unian.ua": "УНІАН",
    "znaj.ua": "Znaj.ua",
    "https://www.liga.net": "LIGA.net",
    "112.ua": "112.ua",
    "https://www.segodnya.ua/ua": "Сьогодні",
    "apostrophe.ua": "Апостроф",
    "www.unn.com.ua": "УНН",
    "https://ukranews.com": "Українські новини",
    "https://www.epravda.com.ua": "Економічна Правда",
    "espreso.tv": "Еспресо ТВ",
    "www.radiosvoboda.org": "Радіо Свобода",
    "fakty.com.ua": "Факти ICTV",
    "https://ukr.lb.ua": "LB.ua",
    "https://hromadske.ua/": "Громадське",
    "zik.ua": "ZIK",
    "https://dt.ua/": "Дзеркало тижня",
    "focus.ua": "Фокус",
    "https://ua.interfax.com.ua": "Інтерфакс-Україна",
    'https://ua.korrespondent.net/': "Корреспондент.net",
    'glavcom.ua': "Главком", 
    'vgolos.com.ua': "Вголос",
    'nv.ua': "НВ",
    'suspilne.media': "Суспільне",
    'zaxid.net': "Zaxid.net",
    'babel.ua': "Бабель",
    'gazeta.ua': "Gazeta.ua",
    'ukrinform.ua': "Укрінформ"
    }

news['domain_alias'] = news.domain.apply(lambda x: alias[x])

In [ ]:
def set_language(news):
    ru = ['https://ukranews.com', 'focus.ua', 'strana.ua', 'https://censor.net.ua', 'nv.ua']
    
    news['lang'] = None
    
    news.lang.mask(news.text.str.contains(r"[іїІЇЄҐґє]", na=False), 'uk', inplace=True)
    news.lang.mask(news.domain.str.match(r"|".join(ru)), 'ru', inplace=True)
    news.lang.mask(news.text.str.contains(r"([ЫыЁёЭэЪъ].*){5,}", flags=re.S, na=False), 'ru', inplace=True)
    news.lang.mask(news.text.str.contains(r"([іїІЇЄҐґє].*){10,}", flags=re.S, na=False), 'uk', inplace=True)
    
    news.lang.mask(((news.domain=='https://www.liga.net')&(news.lang.isna())), 'ru', inplace=True)
    
    news.lang.mask((news.lang.isna())&(news.text.str.contains(r"([іїІЇЄҐґє].*){3,}", flags=re.S, na=False)), 'uk', inplace=True)
    news.lang.mask((news.lang.isna())&(news.text.str.contains(r"([ЫыЁёЭэЪъ].*){3,}", flags=re.S, na=False)), 'ru', inplace=True)
    
    news.lang.mask((news.lang.isna())&(news.title.str.contains(r"([іїІЇЄҐґє].*){1,}", flags=re.S, na=False)), 'uk', inplace=True)
    news.lang.mask((news.lang.isna())&(news.title.str.contains(r"([ЫыЁёЭэЪъ].*){1,}", flags=re.S, na=False)), 'ru', inplace=True)
    
    for i, r in news[news.lang.isna()&news.text.notna()].iterrows():
        try:
            if detect(r.text)=='en':
                news.lang.at[i] = 'en'
        except:
            pass   

In [ ]:
set_language(news)